In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


RendererRegistry.enable('default')

In [35]:
import altair as alt
from embeddings_analysis import EmbeddingsLoader

from sklearn.base import clone
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE
from umap import UMAP


alt.data_transformers.disable_max_rows()
alt.renderers.set_embed_options(
    theme='dark'
)

RendererRegistry.enable('default')

# OLMo-2-1124-7B

The model has been chosen as one of the targets of this analysis because of its inclination towards research and hackability. Like the other models considered, OLMo uses a BPE tokenizer in which the 0-999 range seems to be hardcoded to be encoded with a single token for each number.
Only numbers in this range are considered, even though there might be bigger integers that get encoded with a single token by the BPE tokenizer.

It is also notable that the OLMo model shares a lot of similarities with the LLaMa

We check the numbers that get encoded in a single embedding vector by running the tokenizer on all the numbers in the range until we find the first one that gets encoded with more than one token.

In [2]:
model_id = "allenai/OLMo-2-1124-7B"
loader = EmbeddingsLoader(model_id)
loader.smallest_multitoken_number()

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

1000

One of the goal of this analysis is to find structures in the embeddings.

Hypotheses:
- The representation of different models converges to similar structures
- Numerical embeddings have a representation that favors numerical calculation tasks
- There are structures that the embeddings converge towards in the pursuit of certain tasks
    - Some structure provide affordances that allow for better resolution of certain tasks.

It's also notable that the choice of having specific tokens for the numbers in the range 0-999 bias the model toward a direct representation of positive integers, possibly negating symmetries with negative numbers.

TODO Confront this with research about model convergence

In [3]:
# Loading the number embeddings and 1000 random embeddings for comparison

number_embeddings = loader.numbers()
random_embeddings = loader.random()

Dimensionality reduction techniques are employed to visualize the structures that might emerge from the embeddings. They also are compared to a visualization of random embeddings to show that the structure is specific to the number embeddings.

# Principal Component Analysis

The numerical embeddings form a clear curve, suggesting they might follow a meaningful geometric pattern. The structure might follow this pattern for different reasons:

- The embdeddings capture non-linear relationships between the number tokens, which may take place in natural language data.
- PCA tries to preserve large distances in the data, which can cause a "bending" of inherently sequential data when projected to lower dimensions.
- Curves in PCA might happen because of statistical phenomena that should be 
Guttman effect, see [Camiz](https://www.researchgate.net/publication/228760485_The_Guttman_effect_Its_interpretation_and_a_new_redressing_method)

The color gradient is smooth, showing that the embedding space captures numerical proximity. Looking at the top right part of the curve, there looks to be a smear. It might seem incidental, but I'm gonna argue with further visualizations that it represents a recursive encoding of the numbers with one and two digits in the embedding space. Lower numbers find themselves in the right part of the color gradient, and they also happen to be in the right place 

In [ ]:
number_dimr = number_embeddings.dim_reduction(PCA(n_components=100))
random_pca = random_embeddings.dim_reduction(PCA(n_components=100))

alt.hconcat(number_dimr.plot(), random_pca.plot()).resolve_scale(color="independent")

alt.HConcatChart(...)

In [84]:
number_svd = number_embeddings.dim_reduction(TruncatedSVD(n_components=100))
random_svd = random_embeddings.dim_reduction(TruncatedSVD(n_components=100))

alt.hconcat(number_svd.plot(), random_svd.plot()).properties().resolve_scale(color="independent")

alt.HConcatChart(...)

Bla bla bla I'm so smart

In [ ]:
number_svd.plot_digit_overview()

alt.VConcatChart(...)

In [93]:
number_svd.plot_correlation_heatmap()

AttributeError: 'EmbeddingsDimReduction' object has no attribute 'plot_correlation_heatmap'

Here are shown other representations for reference, even though PCA seems to be the best one as it gives a striking visual representation using only linear transformations. The other methods tried where t-SNE and UMAP, which in this particular case give the impression of an organized structure even in the random embeddings. There might be interesting observations to make about the digit encoding in this case.

In [81]:
tsne_args = dict(perplexity=75, max_iter=3000, learning_rate=500, early_exaggeration=20, random_state=42)

number_tsne = number_embeddings.dim_reduction(TSNE(**tsne_args))
random_tsne = random_embeddings.dim_reduction(TSNE(**tsne_args))

alt.hconcat(number_tsne.plot(), random_tsne.plot()).resolve_scale(color="independent")


alt.HConcatChart(...)

In [ ]:
number_tsne.plot_digit_overview()

alt.VConcatChart(...)

In [8]:
alt.hconcat(plots['explained_variance'], plots['cumulative_variance'])

alt.HConcatChart(...)

In [11]:
plots['component_patterns']

alt.Chart(...)